Products_most_reordered

In [ ]:
# Create a dataframe with the most reordered products
op_one=op[op.reordered==1]
products_most_reordered = op_one.groupby('product_id')['reordered'].count().to_frame('most_reordered') #
products_most_reordered = products_most_reordered.reset_index()
products_most_reordered.head()

In [ ]:
prd=prd.merge(products_most_reordered, on='product_id', how='left')

del products_most_reordered
gc.collect()

In [ ]:
prd['most_reordered']=prd['most_reordered'].fillna(value=0)
prd.head()

reorder frequency per customer

In [ ]:
u_reorder = op.groupby('user_id')['reordered'].mean().to_frame('u_reordered_ratio')
u_reorder = u_reorder.reset_index()
u_reorder.head()

In [ ]:
user = user.merge(u_reorder, on='user_id', how='left')

del u_reorder
gc.collect()

user.head()

User product avg pos

In [ ]:
#We calculate the average position of each product for each customer
user_product_avg_pos=op.groupby(['user_id','product_id'])[['add_to_cart_order']].mean()

#we rename the add_to_cart_order
user_product_avg_pos=user_product_avg_pos.rename(columns={'add_to_cart_order':'mean_add_to_cart_order'})
user_product_avg_pos.head()

# Reset the index of the DF so to bring user_id & product_id indices to columns
user_product_avg_pos = user_product_avg_pos.reset_index()

#we calculate the biggest basket - it will be used when we will fill Nan values later
max_basket_size=max(user_product_avg_pos.max())



user_product_avg_pos.head()

times product bought last 5 orders

In [ ]:
prd['order_number_back'] = prd.groupby('user_id')['order_number'].transform(max) - prd.order_number +1 
prd5 = prd[prd.order_number_back <= 5]
last_five = prd5.groupby(['user_id','product_id'])[['order_id']].count()
last_five.columns = ['times_last5']
last_five['times_last5_ratio'] = last_five.times_last5 / 5
prd5_with_five_orders= prd5.groupby('user_id').filter(lambda x: x.order_number_back.max() == 5)
